This notebook is a model builder and traning for batch optimization on RAG Chat internal functions


Imports 

In [319]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy import optimize

Converting training dataset to pandas dataframe and visualizing it

In [258]:
df = pd.read_csv('model_train_v1.csv')
df.head()

,timestamp,batch_size,sentence_amount,total_emd_time (s),total_ind_time (s),avg_emd_time (s),avg_ind_time (s)
0,2024-08-10 21:15:12,1450,1684,5.4117,0.0030,0.0231,0.000013
1,2024-08-10 21:14:46,1450,1326,3.5411,0.0105,0.0027,0.000008
2,2024-08-10 21:14:07,1450,551,2.5528,0.0050,0.0046,0.000009
3,2024-08-10 21:13:48,1450,661,2.8908,0.0066,0.0044,0.000010
4,2024-08-10 21:13:18,1450,2263,7.0622,0.0053,0.0087,0.000007


Slicing the dataset to get only the traning data and creating new features

In [259]:
train_set = df[['sentence_amount','batch_size','total_emd_time (s)']]
train_set

,sentence_amount,batch_size,total_emd_time (s)
0,1684,1450,5.4117
1,1326,1450,3.5411
2,551,1450,2.5528
3,661,1450,2.8908
4,2263,1450,7.0622
...,...,...,...
234,661,1000,2.6770
235,551,1000,2.4963
236,708,1000,2.5886
237,11,1000,0.8007


Splitting independent variables and depentdent variables of traning datase

In [261]:
x = train_set[['sentence_amount','batch_size']] #independent variables
y = train_set[['total_emd_time (s)']] #dependent variables

Splitting dataset to train and test datasets to calculate model metrics

In [262]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)
x_train.shape

(191, 2)

Initializing Linear Regression model from scikit-learn and fitting

In [264]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

Calculating Model R Squared Score


In [316]:
pred = model.predict(x_test)
r2score = r2_score(y_test,pred)
r2score

0.8494302120808415

Calculating Correlation Coefficient of independent and dependent values

In [344]:
array = np.array(y_test).flatten()
coeff=np.corrcoef(pred.flatten(),array)
corr_coeff = coeff[0,1]

Calculating Model Mean squared error

In [266]:
mean_sqr_err = mean_squared_error(y_test,pred)
mean_sqr_err

2.2739613172567554

Creating embedding time prediction function by calling model's prediction method 

In [267]:
def embedding_time_pred(sentence_amount,batch_size):
    a = np.array([[sentence_amount,batch_size]],dtype="object")
    result = model.predict(a)
    return result.flatten()

Creating optimal batch size finder function according to models time predictions

In [268]:
def find_optimal_batch_size(sentence_amount):
    fun = lambda batch_size: embedding_time_pred(sentence_amount,batch_size)
    r = optimize.minimize(fun,x0=[sentence_amount],bounds=[(0,2000)])
    return int(r.x[0])

Created batch_size_calc funtion to determine batch size of the embedding function 

In [347]:
def batch_size_calc(sentence_amount):
    if sentence_amount > 1500 and sentence_amount <= 2000:
        if int(sentence_amount/corr_coeff)>2000:
            return 2000
        else:
            return int(sentence_amount/corr_coeff)
    else:
        return 2000